In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

## Data and imports



In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import lightgbm as lgb
import matplotlib.pyplot as plt

from m5.features import add_special_event_feature, add_snap_feature, add_demand_type_feature
from m5.hierarchy import compute_summing_matrix, get_sales_long, get_prices_long, compute_series_weights, get_sales_long_with_nan
# from rpy2_models.ts_model import TimeSeriesModel, SExpS, Croston, ESX, OES, ExponentialSmoothing
# from rpy2_models import r_models_functions
# from rpy2.robjects import NULL
# from m5.forecast import forecast_bottom_level, get_rmsse, get_wrmsse

from m5.global_model import get_X_and_y
from m5.hierarchy import compute_series_weights


data = Path('data/')
sales = pd.read_csv(data / 'sales_train_validation.csv', dtype={
    'item_id':'category', 'dept_id':'category', 'cat_id':'category', 'store_id':'category', 'state_id':'category'
})
prices = pd.read_csv(data / 'sell_prices.csv', dtype={
    'store_id': 'category', 'item_id': 'category'
})
calendar = pd.read_csv(data / 'calendar.csv', parse_dates=['date'], dtype={
    'weekday':'category', 'd':'category', 'event_name':'category', 'event_type_1':'category', 'event_type_2':'category',
    'snap_CA':'category', 'snap_TX':'category', 'snap_WI':'category'
})

d_cols = [c for c in sales.columns if c.startswith('d_')]
id_cols = [c for c in sales.columns if c not in d_cols]

## [[https://mk0mcompetitiont8ake.kinstacdn.com/wp-content/uploads/2020/02/M5-Competitors-Guide_Final-1.pdf][Competition rules]]



### Interesting details



-   date ranges are 2011-01-29 to 2016-06-19, 1,941 days (5.4 years)
-   In the `calendar.csv` file there are 3 columns for SNAP: for 10 days of the
    month, a subsidy is given to some people to buy some food
-   For each item, the (average) weekly price is given. The price can change over
    time and when it is not available it means that the product was not sold
    during that period (!)



### Evaluation



-   First the RMSSE metrics is computed for each series
    -   This is a modified version of the MASE
-   Then a weighted average (WRMSSE) is calculated.
    -   The weight is calculated by looking at the last 28 days of the training set
        and computing (sum of units sold \* price)
    -   Not only! The WRMSSE is computed by considering errors done at all the
        different aggregation levels (12)
        -   All levels are equally weighted



### Benchmarks



-   Statistical benchmarks methods:
    -   Croston&rsquo;s method: decompose series into non-zero and inter-demand intervals
    -   Temporal aggregation: ADIDA and iMAPA
    -   ESX and ARIMAX, exog features:
        -   Number of states allowing SNAP purchases (0, 1, 2, 3)
        -   Binary for special event
-   ML benchmark methods:
    -   MLP taking last 14 days as input of each single series, 28 hidden nodes and one output node
    -   RF taking last 14 days as input
        -   4 features sampled at each split
    -   Global MLP, learning from last 14 observations of all series
        -   Coefficient of variation of non-zero demands (CV<sup>2</sup>) to distinguish the series
        -   Average number of time periods between 2 successive non-zero demands (ADI)
    -   Global RF, same approach as GMLP
-   ARIMAX and ESX forecast at the topmost level, while global models and local models forecast at the
    lowest level (prouct+store)
    -   To reconcile forecasts, we calculate historical proportions according the
        last 28 days



## Hierarchical time series



In [1]:
%%time
sales_long = get_sales_long(sales)
sum_matrix = compute_summing_matrix(sales)
prices_long = get_prices_long(prices, calendar, sales)

In [1]:
sales_long.to_hdf(data / 'sales_long.h5', key='sales_long')
pd.read_hdf(data / 'sales_long.h5', key='sales_long')

In [1]:
weights = compute_series_weights(sales, prices, calendar)
weights.to_hdf(data / 'weights_all_levels.h5', key='weights_all_levels')

### Grouped time series



Hierarchy state > store > cat > dept > item leads to aggregates by:

-   Total
-   State (3)
-   Store (10)
-   Store + category (30)
-   Store + department (70)
-   Item + Store (30490)

Hierarchy state > category > dept > store:

-   State (3)
-   State + category (9)
-   State + department (21)
-   Store + department (70)

Hierarchy category > department > state > store > item:

-   Category (3)
-   Department (7)
-   Department + State (21)
-   Department + Store (70)
-   Item + Store (30490)

Each product aggregated by:

-   Total (all stores) (3049)
-   State (9147)
-   Store (30490)

    graph TD
    Total-->|3|State
    State-->|10|Store
    State-->|9|State+Category
    State+Category-->|30|Store+Category
    Store-->|30|Store+Category
    Store+Category-->|70|Store+Department
    Store+Department-->|30490|Store+Item
    
    Total-->|3|Category
    Category-->|7|Department
    Department-->|21|Department+State
    Department+State-->|70|Store+Department
    
    Total-->|3049|Item
    Item-->|9147|Item+State
    Item+State-->|30490|Store+Item

![img](/Users/luca/git/experiments/data/kaggle_hierarchy.png)



### sales_long



In [1]:
from typing import Dict, List
import numpy as np
import pandas as pd


all_names = ['level', 'state_id', 'store_id', 'cat_id', 'dept_id', 'item_id']
all_groups = {'total': ['d'],
        'state': ['d', 'state_id'], 'cat': ['d', 'cat_id'], 'item': ['d', 'item_id'],
        'store': ['d', 'state_id', 'store_id'], 'state/cat':['d', 'state_id', 'cat_id'], 'dept': ['d', 'cat_id', 'dept_id'], 'item/state': ['d', 'item_id', 'state_id'],
        'store/cat': ['d', 'state_id', 'store_id', 'cat_id'], 'dept/state': ['d', 'cat_id', 'dept_id', 'state_id'],
        'store/dept': ['d', 'state_id', 'store_id', 'cat_id', 'dept_id']}


def get_sales_long(sales: pd.DataFrame, groups: Dict[str, List[str]] = all_groups, include_bottom_level=True, last_28_days=False):
    d_cols = [c for c in sales.columns if c.startswith('d_')]
    if last_28_days:
        d_cols = d_cols[-28:]

    if groups:
        sales_melted = sales.melt(id_vars=all_names[1:], value_vars=d_cols, var_name='d', value_name='sales')

    all_sales_grouped = []
    for level, group in groups.items():
        sales_grouped = sales_melted.groupby(group, as_index=False).sum()

        missing = [c for c in all_names if c not in group]
        for m in missing:
            sales_grouped.loc[:, m] = 'all'
        sales_grouped.loc[:, 'level'] = level

        sales_grouped = sales_grouped.set_index(['d'] + all_names).unstack(all_names)
        sales_grouped.columns = sales_grouped.columns.droplevel(0)
        all_sales_grouped.append(sales_grouped)

    if include_bottom_level:
        sales_grouped = sales[all_names[1:] + d_cols]
        sales_grouped.loc[:, 'level'] = 'item/store'
        sales_grouped = sales_grouped.set_index(all_names).T
        all_sales_grouped.insert(0, sales_grouped)

    all_sales_grouped = pd.concat(all_sales_grouped, axis=1)

    return all_sales_grouped

In [1]:
import pyarrow as pa
import pyarrow.parquet as pq

# table = pa.Table.from_pandas(df_test)
# pq.write_table(table, 'test.parquet')
table = pq.read_table('/Users/luca/git/experiments/data/sales_long.parquet', use_pandas_metadata=True)

### Summing matrix



[https://otexts.com/fpp2/hts.html](https://otexts.com/fpp2/hts.html)
Matrix having as number of columns the number of bottom series and as number of
rows the total number of time series.
The bottom part is a square matrix with 1 on the diagonal.

$$ \bm{y}_t = \bm{S b}_t $$

At the bottom level there are 70 series describing the store+department level.
There are 10 stores, each of which has 7 departments. Each of these series is represented in
the matrix by a row having one non-zero entry.

The level above contains 30 series for the store+category series. Again, 10
stores, 3 categories. We need to create 30 rows in the matrix. Each row
describes which of the bottom series aggregates to form that store+category entry.



In [1]:
def compute_summing_matrix(sales: pd.DataFrame, groups=all_groups.copy()):
    d_cols = [c for c in sales.columns if c.startswith('d_')]

    sales_grouped = sales.groupby(all_names[1:])[d_cols].sum()
    bottom_level = sales_grouped.index
    sales_grouped['bottom_level_index'] = sales_grouped.reset_index().index

    top_row = pd.DataFrame(data=np.ones((1,sales_grouped.shape[0])), index=['total'], columns=bottom_level)
    groups.pop('total', None)

    bottom_submatrix = pd.DataFrame(data=np.identity(sales_grouped.shape[0]), index=bottom_level, columns=bottom_level)

    central_submatrices = []
    for _, group in groups.items():
        sales_level = sales_grouped.groupby(group[1:])['bottom_level_index'].agg(list)

        level_rows = []
        for _, group in sales_level.iteritems():
            row = np.zeros(sales_grouped.shape[0])
            row[group] = 1
            level_rows.append(row)
        level_submatrix = pd.DataFrame(data=np.vstack(level_rows), index=sales_level.index, columns=bottom_level)
        central_submatrices.insert(0, level_submatrix)

    sum_matrix = pd.concat([top_row] + central_submatrices + [bottom_submatrix], sort=False)

    return sum_matrix

In [1]:
sum_mat = compute_summing_matrix(sales)
sum_mat.shape

In [1]:
groups = {'store': ['d', 'state_id', 'store_id'], 'state/cat':['d', 'state_id', 'cat_id'], 'dept': ['d', 'cat_id', 'dept_id'], 'item/state': ['d', 'item_id', 'state_id']}
sum_mat = compute_summing_matrix(sales, groups)
sum_mat.shape

### Prices



In [1]:
def get_prices_long(prices: pd.DataFrame, calendar: pd.DataFrame, sales: pd.DataFrame, groups: Dict[str, List[str]] = all_groups, include_bottom_level=True, last_28_days=True):
    prices_d = prices.merge(calendar[['wm_yr_wk', 'd']]).drop(columns=['wm_yr_wk'])
    d_cols = [c for c in sales.columns if c.startswith('d_')]
    if last_28_days:
        prices_d = prices_d[prices_d['d'].isin(d_cols[-28:])]
    prices_d = sales[all_names[1:]].merge(prices_d)
    prices_d.loc[:, 'level'] = 'item/store'

    all_prices_h = []
    for level, group in groups.items():
        prices_h = prices_d.groupby(group, as_index=False).sum()

        missing = [c for c in all_names if c not in group]
        for m in missing:
            prices_h.loc[:, m] = 'all'
        prices_h.loc[:, 'level'] = level

        prices_h = prices_h.set_index(['d'] + all_names).unstack(all_names)
        prices_h.columns = prices_h.columns.droplevel(0)
        all_prices_h.append(prices_h)

    if include_bottom_level:
        prices_h = prices_d.set_index(['d'] + all_names).unstack(all_names)
        prices_h.columns = prices_h.columns.droplevel(0)
        all_prices_h.insert(0, prices_h)

    all_prices_h = pd.concat(all_prices_h, axis=1)
    return all_prices_h

### Weights



For each series find its dollar value by multiplying its sales in the last 28 days by
the price it had.
Then, divide it by the total dollar value of the level.



In [1]:
def compute_series_weights(sales: pd.DataFrame, prices: pd.DataFrame, calendar: pd.DataFrame, groups: Dict[str, List[str]] = all_groups, include_bottom_level=True):
    prices_long = get_prices_long(prices, calendar, sales, groups=groups, include_bottom_level=include_bottom_level, last_28_days=True)
    sales_long = get_sales_long(sales, groups=groups, include_bottom_level=include_bottom_level, last_28_days=True)

    assert all(sales_long.columns == prices_long.columns)
    assert all(sales_long.index == prices_long.index)

    dollar_value_by_series = (prices_long * sales_long).sum(axis=0).to_frame(name='series_dollar_value')
    dollar_value_by_level = (prices_long * sales_long).sum(axis=0).sum(level='level').to_frame(name='level_dollar_value')
    dollar_value = dollar_value_by_series.merge(dollar_value_by_level, left_index=True, right_index=True)
    dollar_value.loc[:, 'series_dollar_value_scaled'] = dollar_value['series_dollar_value'] / dollar_value['level_dollar_value']
    assert dollar_value['series_dollar_value_scaled'].sum(level='level').all() == 1

    dollar_value.loc[:, 'series_weight'] = dollar_value['series_dollar_value_scaled'] / (1 + len(groups))
    return dollar_value

In [1]:
groups = {'store': ['d', 'state_id', 'store_id'], 'state/cat':['d', 'state_id', 'cat_id'], 'dept': ['d', 'cat_id', 'dept_id'], 'item/state': ['d', 'item_id', 'state_id']}
weights_df = compute_series_weights(sales_long, prices_long, groups)
weights_df.head()

## Identify when a product was not available



In [1]:
def get_sales_long_with_nan(sales, prices, calendar, groups={}):
    prices_long = get_prices_long(prices, calendar, sales, groups=groups, last_28_days=False)
    sales_long = get_sales_long(sales, groups=groups)
    assert all(sales_long.columns == prices_long.columns)

    prices_long = prices_long.loc[sales_long.index]
    assert all(sales_long.index == prices_long.index)

    sales_long[prices_long.isna()] = pd.NA
    return sales_long

## snap features



In [1]:
import pandas as pd
import numpy as np


def add_snap_feature(calendar: pd.DataFrame):
    snap_cols = [col for col in calendar.columns if col.startswith('snap_')]
    calendar['snap'] = calendar[snap_cols].sum(axis=1)
    return calendar

def add_special_event_feature(calendar: pd.DataFrame):
    calendar['special_event'] = 0
    calendar.loc[calendar['event_type_1'].notna(), 'special_event'] = 1
    return calendar

## Bottom-level models



In [1]:
sales = add_demand_type_feature(sales)
# sales[sales.type == 'smooth'][d_cols].iloc[300, -52:].plot()
one = sales[sales.type == 'smooth'][d_cols].iloc[300:301, :].T

In [1]:
%%time

y_pred_long = forecast_bottom_level(sales, prices, calendar)

### Univariate itermittent models



[https://kourentzes.com/forecasting/wp-content/uploads/2014/05/Petropoulos-Kourentzes-2014-Forecast-Combinations-for-Intermittent-Demand.pdf](https://kourentzes.com/forecasting/wp-content/uploads/2014/05/Petropoulos-Kourentzes-2014-Forecast-Combinations-for-Intermittent-Demand.pdf)



#### R functions



    library(zoo)
    
    intervals <- function(x){
      y<-c()
      k<-1
      counter<-0
      for (tmp in (1:length(x))){
        if(x[tmp]==0){
          counter<-counter+1
        }else{
          k<-k+1
          y[k]<-counter
          counter<-1
        }
      }
      y<-y[y>0]
      y[is.na(y)]<-1
      y
    }
    demand <- function(x){
      y<-x[x!=0]
      y
    }
    SexpS <- function(x, h){
      a <- optim(c(0), SES, x=x, h=1, job="train", lower = 0.1, upper = 0.3, method = "L-BFGS-B")$par
      y <- SES(a=a, x=x, h=1, job="forecast")$mean
      forecast <- rep(as.numeric(y), h)
      return(forecast)
    }
    SES <- function(a, x, h, job){
      y <- c()
      y[1] <- x[1] #initialization
    
      for (t in 1:(length(x))){
        y[t+1] <- a*x[t]+(1-a)*y[t]
      }
    
      fitted <- head(y,(length(y)-1))
      forecast <- rep(tail(y,1),h)
      if (job=="train"){
        return(mean((fitted - x)^2))
      }else if (job=="fit"){
        return(fitted)
      }else{
        return(list(fitted=fitted,mean=forecast))
      }
    }
    Croston <- function(x, h, type){
      if (type=="classic"){
        mult <- 1
        a1 = a2 <- 0.1
      }else if (type=="optimized"){
        mult <- 1
        a1 <- optim(c(0), SES, x=demand(x), h=1, job="train", lower = 0.1, upper = 0.3, method = "L-BFGS-B")$par
        a2 <- optim(c(0), SES, x=intervals(x), h=1, job="train", lower = 0.1, upper = 0.3, method = "L-BFGS-B")$par
      }else if (type=="sba"){
        mult <- 0.95
        a1 = a2 <- 0.1
      }
      yd <- SES(a=a1, x=demand(x), h=1, job="forecast")$mean
      yi <- SES(a=a2, x=intervals(x), h=1, job="forecast")$mean
      forecast <- rep(as.numeric(yd/yi), h)*mult
      return(forecast)
    }
    TSB <- function(x, h){
      n <- length(x)
      p <- as.numeric(x != 0)
      z <- x[x != 0]
    
      a <- c(0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.8)
      b <- c(0.01,0.02,0.03,0.05,0.1,0.2,0.3)
      MSE <- c() ; forecast <- NULL
      for (atemp in a){
        for (btemp in b){
          zfit <- vector("numeric", length(x))
          pfit <- vector("numeric", length(x))
          zfit[1] <- z[1] ; pfit[1] <- p[1]
    
          for (i in 2:n) {
            pfit[i] <- pfit[i-1] + atemp*(p[i]-pfit[i-1])
            if (p[i] == 0) {
              zfit[i] <- zfit[i-1]
            }else {
              zfit[i] <- zfit[i-1] + btemp*(x[i]-zfit[i-1])
            }
          }
          yfit <- pfit * zfit
          forecast[length(forecast)+1] <- list(rep(yfit[n], h))
          yfit <- c(NA, head(yfit, n-1))
          MSE <- c(MSE, mean((yfit-x)^2, na.rm = T) )
        }
      }
      return(forecast[[which.min(MSE)]])
    }
    ADIDA <- function(x, h){
      al <- round(mean(intervals(x)),0) #mean inter-demand interval
      #Aggregated series (AS)
      AS <- as.numeric(na.omit(as.numeric(rollapply(tail(x, (length(x) %/% al)*al), al, FUN=sum, by = al))))
      forecast <- rep(SexpS(AS, 1)/al, h)
      return(forecast)
    }
    iMAPA <- function(x, h){
      mal <- round(mean(intervals(x)),0)
      frc <- NULL
      for (al in 1:mal){
        frc <- rbind(frc, rep(SexpS(as.numeric(na.omit(as.numeric(rollapply(tail(x, (length(x) %/% al)*al), al, FUN=sum, by = al)))), 1)/al, h))
      }
      forecast <- colMeans(frc)
    return(forecast)
    }

    x <- c(1,0,3,4,5,23,3,4)
    iMAPA(x, 2)



#### rpy2 functions



In [1]:
<<rpy2_r_functions>>

r(
"""
    library(zoo)


    intervals <- function(x){
    y<-c()
    k<-1
    counter<-0
    for (tmp in (1:length(x))){
        if(x[tmp]==0){
        counter<-counter+1
        }else{
        k<-k+1
        y[k]<-counter
        counter<-1
        }
    }
    y<-y[y>0]
    y[is.na(y)]<-1
    y
    }

    demand <- function(x){
    y<-x[x!=0]
    y
    }

    SES <- function(a, x, h, job){
    y <- c()
    y[1] <- x[1] #initialization

    for (t in 1:(length(x))){
        y[t+1] <- a*x[t]+(1-a)*y[t]
    }

    fitted <- head(y,(length(y)-1))
    forecast <- rep(tail(y,1),h)
    if (job=="train"){
        return(mean((fitted - x)^2))
    }else if (job=="fit"){
        return(fitted)
    }else{
        return(list(fitted=fitted,mean=forecast))
    }
    }

    """
)


sexps = r("""
function(x, h){
  a <- optim(c(0), SES, x=x, h=1, job="train", lower = 0.1, upper = 0.3, method = "L-BFGS-B")$par
  y <- SES(a=a, x=x, h=1, job="forecast")$mean
  forecast <- rep(as.numeric(y), h)
  return(forecast)
}
""")

croston = r("""
function(x, h, type){
    if (type=="classic"){
        mult <- 1
        a1 = a2 <- 0.1
    }else if (type=="optimized"){
        mult <- 1
        a1 <- optim(c(0), SES, x=demand(x), h=1, job="train", lower = 0.1, upper = 0.3, method = "L-BFGS-B")$par
        a2 <- optim(c(0), SES, x=intervals(x), h=1, job="train", lower = 0.1, upper = 0.3, method = "L-BFGS-B")$par
    }else if (type=="sba"){
        mult <- 0.95
        a1 = a2 <- 0.1
    }
    yd <- SES(a=a1, x=demand(x), h=1, job="forecast")$mean
    yi <- SES(a=a2, x=intervals(x), h=1, job="forecast")$mean
    forecast <- rep(as.numeric(yd/yi), h)*mult
    return(forecast)
}
""")

tsb = r("""
function(x, h){
  n <- length(x)
  p <- as.numeric(x != 0)
  z <- x[x != 0]

  a <- c(0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.8)
  b <- c(0.01,0.02,0.03,0.05,0.1,0.2,0.3)
  MSE <- c() ; forecast <- NULL
  for (atemp in a){
    for (btemp in b){
      zfit <- vector("numeric", length(x))
      pfit <- vector("numeric", length(x))
      zfit[1] <- z[1] ; pfit[1] <- p[1]

      for (i in 2:n) {
        pfit[i] <- pfit[i-1] + atemp*(p[i]-pfit[i-1])
        if (p[i] == 0) {
          zfit[i] <- zfit[i-1]
        }else {
          zfit[i] <- zfit[i-1] + btemp*(x[i]-zfit[i-1])
        }
      }
      yfit <- pfit * zfit
      forecast[length(forecast)+1] <- list(rep(yfit[n], h))
      yfit <- c(NA, head(yfit, n-1))
      MSE <- c(MSE, mean((yfit-x)^2, na.rm = T) )
    }
  }
  return(forecast[[which.min(MSE)]])
}
""")

adida = r("""
function(x, h){
  al <- round(mean(intervals(x)),0) #mean inter-demand interval
  #Aggregated series (AS)
  AS <- as.numeric(na.omit(as.numeric(rollapply(tail(x, (length(x) %/% al)*al), al, FUN=sum, by = al))))
  forecast <- rep(SexpS(AS, 1)/al, h)
  return(forecast)
}
""")

imapa = r("""
function(x, h){
  mal <- round(mean(intervals(x)),0)
  frc <- NULL
  for (al in 1:mal){
    frc <- rbind(frc, rep(SexpS(as.numeric(na.omit(as.numeric(rollapply(tail(x, (length(x) %/% al)*al), al, FUN=sum, by = al)))), 1)/al, h))
  }
  forecast <- colMeans(frc)
  return(forecast)
}
""")

#### rpy2 models



In [1]:
<<rpy2_models>>

class SExpS(TimeSeriesModel):
    def fit_predict(
        self,
        y_train: np.ndarray,
        y_test: pd.Series,
        xreg_train: Union[np.ndarray, NULLType],
        xreg_test: Union[np.ndarray, NULLType],
    ) -> DataFrame:
        try:
            y_hat = r_models_functions.sexps(y_train, y_test.shape[0])
        except Exception:
            raise ModelNotFitError
        df_forecast = DataFrame(y_hat, columns=[y_test.name], index=y_test.index)
        return df_forecast

class Croston(TimeSeriesModel):
    def __init__(self, type='classic'):
        super().__init__()
        self.type = type

    def fit_predict(
        self,
        y_train: np.ndarray,
        y_test: pd.Series,
        xreg_train: Union[np.ndarray, NULLType],
        xreg_test: Union[np.ndarray, NULLType],
    ) -> DataFrame:
        try:
            y_hat = r_models_functions.croston(y_train, y_test.shape[0], self.type)
        except Exception:
            raise ModelNotFitError
        df_forecast = DataFrame(y_hat, columns=[y_test.name], index=y_test.index)
        return df_forecast

class TSB(TimeSeriesModel):
    def fit_predict(
        self,
        y_train: np.ndarray,
        y_test: pd.Series,
        xreg_train: Union[np.ndarray, NULLType],
        xreg_test: Union[np.ndarray, NULLType],
    ) -> DataFrame:
        try:
            y_hat = r_models_functions.tsb(y_train, y_test.shape[0])
        except Exception:
            raise ModelNotFitError
        df_forecast = DataFrame(y_hat, columns=[y_test.name], index=y_test.index)
        return df_forecast

class ADIDA(TimeSeriesModel):
    def fit_predict(
        self,
        y_train: np.ndarray,
        y_test: pd.Series,
        xreg_train: Union[np.ndarray, NULLType],
        xreg_test: Union[np.ndarray, NULLType],
    ) -> DataFrame:
        try:
            y_hat = r_models_functions.adida(y_train, y_test.shape[0])
        except Exception:
            raise ModelNotFitError
        df_forecast = DataFrame(y_hat, columns=[y_test.name], index=y_test.index)
        return df_forecast

class iMAPA(TimeSeriesModel):
    def fit_predict(
        self,
        y_train: np.ndarray,
        y_test: pd.Series,
        xreg_train: Union[np.ndarray, NULLType],
        xreg_test: Union[np.ndarray, NULLType],
    ) -> DataFrame:
        try:
            y_hat = r_models.imapa(y_train, y_test.shape[0])
        except Exception:
            raise ModelNotFitError
        df_forecast = DataFrame(y_hat, columns=[y_test.name], index=y_test.index)
        return df_forecast

### CV2, ADI features and classification



• Smooth demand: regular demand over time with a limited vari- ation in quantity;
• Intermittent demand: extremely sporadic demand, with no ac- centuated
variability in the quantity of the single demand;
• Erratic demand: regular distribution over time, but large varia- tion in
quantity;
• Lumpy demand: extremely sporadic demand, great number of zero-demand periods and large variation in quantity

According to Kostenko and Hyndman(2006), an approximate rule for selecting
Croston’s methodover SBA is given by v≤2−(3/2)p,where v denotes the square of the
coefficient of variation of the demandsand p refers to the meanvalue of the
intervals. Graphs (e) through (h) refer to the SBC-KH-SES, where we propose theuse of SES in the casep= 1, irrelevant of the value ofv.

-   MAPA produces forecast by averaging forecasts at different frequencies
    (aggregation level of 8 periods) (good for longer horizon?)

if smooth -> SES
if lumpy or intermittent -> SBA
if erratic -> croston



In [1]:
def add_cv2_feature(sales: pd.DataFrame):
    d_cols = [c for c in sales.columns if c.startswith('d_')]
    _sales = sales.replace({0: pd.NA})
    sales['cv2'] = (_sales[d_cols].std(axis=1) / _sales[d_cols].mean(axis=1)) ** 2
    return sales

def add_adi_feature(sales: pd.DataFrame):
    d_cols = [c for c in sales.columns if c.startswith('d_')]
    sales['adi'] = sales.shape[1] / np.count_nonzero(sales[d_cols], axis=1)
    return sales

def add_demand_type_feature(sales: pd.DataFrame):
    sales = add_cv2_feature(sales)
    sales = add_adi_feature(sales)

    sales.loc[(sales['adi']>4/3) & (sales['cv2']>0.5), 'type'] = 'lumpy'
    sales.loc[(sales['adi']>4/3) & (sales['cv2']<=0.5), 'type'] = 'intermittent'
    sales.loc[(sales['adi']<=4/3) & (sales['cv2']>0.5), 'type'] = 'erratic'
    sales.loc[(sales['adi']<=4/3) & (sales['cv2']<=0.5), 'type'] = 'smooth'
    return sales

### Forecast all bottom level series



In [1]:
import pandas as pd
import numpy as np

from m5.features import add_demand_type_feature
from m5.hierarchy import get_sales_long_with_nan, all_names
from rpy2_models.ts_model import SExpS, Croston
from rpy2_models import r_models_functions


def get_y_train_val(series: pd.Series, val_days=28):
    y_train = series.dropna()
    y_val = series.iloc[-val_days:]
    return y_train, y_val


def forecast_bottom_level(sales: pd.DataFrame, prices: pd.DataFrame, calendar: pd.DataFrame):
    sales_with_type = add_demand_type_feature(sales.copy())
    sales_with_type = sales_with_type.set_index(all_names[1:])

    sales_long = get_sales_long_with_nan(sales, prices, calendar)

    y_pred_long = []
    for _, series in sales_long.iteritems():
        series_name = series.name[1:]
        series_type = sales_with_type.loc[series_name]['type']
        if series_type in ['lumpy', 'intermittent']:
            model = Croston(type='sba')
        elif series_type == 'erratic':
            model = Croston(type='optimized')
        else:
            model = SExpS()

        y_train, y_val = get_y_train_val(series)
        y_pred = model.fit_predict(y_train.values, y_val, None, None)
        y_pred_long.append(y_pred)

    y_pred_long = pd.concat(y_pred_long, axis=1)
    return y_pred_long

### occurence exp smoothing



In [1]:
r("library(smooth)")

fit_oes = r("""
    function(y_train, xreg=NULL){
        ts(y_train, frequency=7) %>%
            oes(model="YYY", occurence="auto", xreg=xreg)
    }
""")

predict_oes = r("""
    function(model, h, xreg=NULL){
        forecast(model, h=h, xreg=xreg, interval="n") %>%
            data.frame()
    }
""")

In [1]:
class OES(TimeSeriesModel):
    def fit(self, y_train: np.ndarray, xreg_train: np.ndarray) -> TimeSeriesModel:
        self.model = r_models_functions.fit_oes(y_train, xreg_train)
        self.is_fitted_ = True
        self.fitted = r_models_functions.get_fitted_values(self.model)
        return self.model

    def predict(self, horizon_max: int, xreg_test: np.ndarray) -> pd.DataFrame:
        check_is_fitted(self, "is_fitted_")
        forecast = r_models_functions.predict_oes(self.model, horizon_max, xreg_test)
        forecast = pd.DataFrame(forecast['Point.Forecast'], columns=['prediction'])
        return forecast

In [1]:
from rpy2.robjects import NULL

model = OES()
model.fit(one.values, NULL)
model.predict(28, NULL)

### Ensembling



&ldquo;Let us consider thesimplest case where the point forecasts of two methods are
combined with equal weights.In orderfor this to improve forecasting performance,
the point forecasts of the two methods should lie in theopposite sides of the
future actual values. Otherwise, if the point forecasts are both lower or
greaterthan the actual, then the final forecast error is equal
to the simple average of the forecast errorof the two methods.
In the majority of cases, these demand rate forecasts are collectively
eitherunderestimating the observed demand when a non-zero demand is recorded, or
overestimating it,when the observed
demand is zero due to the intermittent nature of the data&rdquo;



## Top-level univariate models



In [1]:
groups = {'total': ['d']}

sales_long = get_sales_long(sales, groups, include_bottom_level=False)
sales_long.shape

In [1]:
calendar = add_snap_feature(calendar)
calendar = add_special_event_feature(calendar)

features = ['snap', 'special_event']
input = sales_long.merge(calendar[['d'] + features].set_index('d'), left_index=True, right_index=True)

y, X = input.iloc[:, 0], input[features]

y_train, X_train = y.iloc[:-28], X.iloc[:-28 :]
y_val, X_val = y.iloc[-28:], X.iloc[-28:, :]

### pmdarima



In [1]:
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import BoxCoxEndogTransformer
import pmdarima as pm

pipeline = Pipeline([
    ("boxcox", BoxCoxEndogTransformer()),
    ("model", pm.AutoARIMA(m=7, seasonal=True, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True))
])

pipeline.fit(y=y_train, exogenous=X_train)
y_pred = pipeline.predict(n_periods=X_val.shape[0], exogenous=X_val)

fig, ax = plt.subplots(figsize=(20,10))
pd.DataFrame(data=y_pred, columns= ['pred'], index=X_val.index).plot(ax=ax)
y_val.to_frame().plot(ax=ax)

### rpy2 auto.arima



In [1]:
from rpy2_models.ts_model_cv import ARIMAXCV

In [1]:
model = ARIMAXCV(seasonal=True)
model.find_model_structure(y_train = y_train.values, xreg_train=X_train.values)

In [1]:
y_pred = model.predict(horizon_max = X_val.shape[0], xreg_test = X_val.values)

fig, ax = plt.subplots(figsize=(20,10))
y_pred[['prediction']].plot(ax=ax)
y_val.to_frame().plot(ax=ax)

### exponential smoothing with exogenous variables



In [1]:
r("library(smooth)")

fit_esx = r("""
    function(y_train, xreg){
        ts(y_train, frequency=7) %>%
            es(xreg, occurence="auto")
    }
""")

predict_esx = r("""
    function(model, h, xreg){
        forecast(model, h=h, xreg=xreg, interval='n') %>%
            data.frame()
    }
""")

In [1]:
class ESX(TimeSeriesModel):
    def fit(self, y_train: np.ndarray, xreg_train: np.ndarray) -> TimeSeriesModel:
        self.model = r_models_functions.fit_esx(y_train, xreg_train)
        self.is_fitted_ = True
        self.fitted = r_models_functions.get_fitted_values(self.model)
        return self.model

    def predict(self, horizon_max: int, xreg_test: np.ndarray) -> pd.DataFrame:
        check_is_fitted(self, "is_fitted_")
        forecast = r_models_functions.predict_esx(self.model, horizon_max, xreg_test)
        forecast = pd.DataFrame(forecast['Point.Forecast'], columns=['prediction'])
        return forecast

In [1]:
model = ESX()
model.fit(y_train, X_train.values)
y_pred = model.predict(horizon_max = X_val.shape[0], xreg_test = X_val.values)
y_pred

In [1]:
fig, ax = plt.subplots(figsize=(20,10))
y_pred[['prediction']].plot(ax=ax)
y_val.to_frame().plot(ax=ax)

[0m



### prophet



In [1]:
from fbprophet import Prophet

df = sales_long.iloc[:-28, 0:1].merge(calendar[['d', 'date']].set_index('d'), left_index=True, right_index=True)
df.columns = ['y', 'ds']
m = Prophet()
m.fit(df)

future = m.make_future_dataframe(periods=28, include_history=False)
forecast = m.predict(future)

In [1]:
fig, ax = plt.subplots(figsize=(20,10))
forecast[['ds', 'yhat']].set_index('ds').plot(ax=ax)
pd.DataFrame(sales_long.iloc[-28:, 0].values, index=forecast['ds']).plot(ax=ax)

### try auto-ces/ssarima/smoothCombine



### try ves (and covar)



## Global models



### Features



In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import List


def get_sales_and_price_wide(sales, prices, calendar):
    # map days to dates
    day_to_date = calendar.set_index("d")["date"].to_dict()
    sales_wide = sales.rename(columns=day_to_date)
    # select dates
    all_dates = sales_wide.select_dtypes('int').columns
    sales_wide = sales_wide.set_index(['item_id', 'store_id'])[all_dates]
    sales_wide.columns = pd.to_datetime(sales_wide.columns).rename('date')
    # add date to prices
    prices_wide = prices.merge(calendar[['date', 'wm_yr_wk']]).drop(columns=['wm_yr_wk']).set_index(['date', 'item_id', 'store_id']).unstack(level='date').sort_index()
    prices_wide.columns = prices_wide.columns.droplevel(0)
    prices_wide.columns = pd.to_datetime(prices_wide.columns).rename('date')
    # make sure prices_wide and sales_wide have same index
    # TODO what about prices for future dates?
    prices_wide = prices_wide.loc[prices_wide.index.intersection(sales_wide.index), sales_wide.columns.intersection(sales_wide.columns)]
    # set demand to NA in dates where a product was not available
    sales_wide[prices_wide.isna()] = pd.NA

    return prices_wide, sales_wide

def compute_float_features(sales_wide, prices_wide, forecast_horizon = 28):
    all_features_wide = {}
    for lag in tqdm([0, 7, 28], "autoregressive features"):
        autoregressive = sales_wide.shift(forecast_horizon + lag, axis=1)
        all_features_wide[f"ar_{lag}"] = autoregressive

    for window in tqdm([7, 30, 60, 90, 180], "rolling features"):
        mean = sales_wide.shift(forecast_horizon, axis=1).rolling(window, axis=1).mean()
        all_features_wide[f"mean_{window}"] = mean

        std = sales_wide.shift(forecast_horizon, axis=1).rolling(window, axis=1).std()
        all_features_wide[f"std_{window}"] = std

        ewma = sales_wide.shift(forecast_horizon, axis=1).ewm(span=window, min_periods=window).mean()
        all_features_wide[f"ewma_{window}"] = ewma

    for window in tqdm([30], "skew, kurt features"):
        skew = sales_wide.shift(forecast_horizon, axis=1).rolling(window, axis=1).skew()
        all_features_wide[f"skew_{window}"] = skew

        kurt = sales_wide.shift(forecast_horizon, axis=1).rolling(window, axis=1).kurt()
        all_features_wide[f"kurt_{window}"] = kurt

    for window in tqdm([7, 30], "price std features"):
        price_std = prices_wide.rolling(window, axis=1).std()
        all_features_wide[f"price_std_{window}"] = price_std

    for window in tqdm([365], "price change features"):
        price_max_year = prices_wide.rolling(window, axis=1).max()
        price_change_year = (prices_wide - price_max_year) / price_max_year
        price_change_week = (prices_wide - prices_wide.shift(1, axis=1)) / prices_wide.shift(1, axis=1)

        all_features_wide[f"price_max_365"] = price_max_year
        all_features_wide[f"price_change_365"] = price_change_year
        all_features_wide[f"price_change_7"] = price_change_week

    return float_features_wide

def get_train_val_dates(float_features_wide, val_days, only_val):
    train_val_dates = list(float_features_wide.values())[0].columns
    # only include dates for which all features have at least one observation
    for _, f in tqdm(float_features_wide.items(), "computing train and validation index"):
        train_val_dates = train_val_dates.intersection(f.dropna(axis=1, how='all').columns)

    train_dates = train_val_dates[:-val_days] if not only_val else pd.DatetimeIndex([])
    val_dates = train_val_dates[-val_days:]

    return train_dates, val_dates

def stack_float_features(float_features_wide, train_dates, val_dates):
    train_val_dates = train_dates.union(val_dates)

    float_features_names = [*float_features_wide.keys()]
    all_features_long = []
    for name in tqdm(float_features_names, "building design matrix"):
        f = float_features_wide.pop(name)  # casting before stacking is efficient
        f = f.loc[:, train_val_dates].astype(np.float32).stack(dropna=False).sort_index() # sort index so we are sure all features are aligned
        all_features_long.append(f.values[:, np.newaxis]) # convert 1D array to column vector
    all_features_long = pd.DataFrame(data=np.hstack(all_features_long), index=f.index, columns=float_features_names)
    all_features_long.index = all_features_long.index.rename('date', level=-1)

    return all_features_long

def add_datetime_features(train_dates, val_dates, calendar):
    train_val_dates = train_dates.union(val_dates)

    datetime_features = calendar[['date', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI']].set_index('date').loc[train_val_dates]

    datetime_attrs = [
        "quarter",
        "month",
        "week",
        "day",
        "dayofweek",
        "is_year_end",
        "is_year_start",
        "is_quarter_end",
        "is_quarter_start",
        "is_month_end",
        "is_month_start",
    ]
    for attr in tqdm(datetime_attrs, "datetime features"):
        datetime_features.loc[:, attr] = getattr(datetime_features.index, attr)
        # cast int features to int8, the others are categorical
        if not attr.startswith('is'):
            datetime_features[attr] = datetime_features[attr].astype('int8')
    datetime_features.loc[:, "is_weekend"] = datetime_features.index.dayofweek.isin([5, 6])
    datetime_features.loc[:, "year"] = datetime_features.index.year

    return datetime_features

def get_X_and_y(sales: pd.DataFrame, prices: pd.DataFrame, calendar: pd.DataFrame, cat_features: List[str], val_days=28, only_val=False):
    sales_wide, prices_wide = get_sales_and_price_wide(sales, prices, calendar)

    float_features_wide = compute_float_features(sales_wide, prices_wide)

    train_dates, val_dates = get_train_val_dates(float_features_wide, val_days, only_val)
    # test_dates = prices_wide.iloc[:, -28:]

    all_features_long = stack_float_features(float_features_wide, train_dates, val_dates)

    # set index as columns to add item_id, store_id and date
    all_features_long = all_features_long.assign(**all_features_long.index.to_frame())

    datetime_features = add_datetime_features(train_dates, val_dates, calendar)
    all_features_long = all_features_long.merge(datetime_features, left_index=True, right_index=True)
    all_features_long["date"] = all_features_long["date"].astype('int')

    # cast categories
    all_features_long[cat_features] = all_features_long[cat_features].astype('category')

    # TODO add function that computes X_val and X_test by shifting with the forecast horizon
    # TODO dropna on num features
    sales_long = sales_wide.loc[:, train_dates.union(val_dates)].stack(dropna=False).sort_index()
    sales_long.index = sales_long.index.rename('date', level=-1)

    y_train, X_train = sales_long.loc(axis=0)[:, :, train_dates], all_features_long.loc(axis=0)[:, :, train_dates]
    y_val, X_val = sales_long.loc(axis=0)[:, :, val_dates], all_features_long.loc(axis=0)[:, :, val_dates]
    # X_test = all_features_long.loc(axis=0)[:, :, test_dates]

    return (X_train, y_train), (X_val, y_val)#, X_test

### more features



In [1]:
def get_ratio_features(df):

    df['LastWeekByPrevWeek'] = get_ratio(df['LastWeekValues'],df['PrevWeekValues'])
    df['Last2WeeksAverage'] = ((df['LastWeekValues']+df['PrevWeekValues'])/2.0).fillna(df['LastWeekValues'])
    df['LastWeekByForecastMean'] = get_ratio(df['LastWeekValues'],df['MeanByForecastId'])
    df['LastWeekBySiteMean'] = get_ratio(df['LastWeekValues'],df['MeanBySiteId'])
    df['ForecastMeanBySiteMean'] = get_ratio(df['MeanByForecastId'],df['MeanBySiteId'])
    df['ForecastMedianBySiteMedian'] = get_ratio(df['MedianByForecastId'],df['MedianBySiteId'])
    df['LastWeekBySiteMax'] = get_ratio(df['LastWeekValues'], df['MaxBySiteId'])
    df['LastWeekBySiteMin'] = get_ratio(df['LastWeekValues'], df['MinBySiteId'])
    df['SiteMaxBySiteMin'] = get_ratio(df['MaxBySiteId'], df['MinBySiteId'])
    df['SiteMeanBySiteMedian'] = get_ratio(df['MeanBySiteId'], df['MedianBySiteId'])
    df['LastDayByPrev'] = get_ratio(df['LastDayValues'], df['MeanBySiteIdTime1D_offset1W'])
    df['LastHourByPrev'] = get_ratio(df['MeanBySiteId1h'], df['MeanBySiteId1h_offset1W'])
    df['CurTempMinusMonthMedian'] = (df['Temperature'] - df['MedianTemperatureBySiteIdTime1M']).fillna(0)

#    df['LastDayBySiteMean'] = get_ratio(df['LastDayValues'],df['MeanBySiteId10Y'])
#    df['LastHourBySiteMean'] = get_ratio(df['MeanBySiteId1h'],df['MeanBySiteId10Y'])



    return df

### tpot



In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from tpot import TPOTRegressor
from joblib import dump

import sys
sys.path.append('src')
from m5.global_model import compute_features, get_X_y

data = Path('data/')
sales = pd.read_csv(data / 'sales_train_validation.csv', dtype={
    'item_id':'category', 'dept_id':'category', 'cat_id':'category', 'store_id':'category', 'state_id':'category'
})
prices = pd.read_csv(data / 'sell_prices.csv', dtype={
    'store_id': 'category', 'item_id': 'category'
})
calendar = pd.read_csv(data / 'calendar.csv', parse_dates=['date'], dtype={
    'weekday':'category', 'd':'category', 'event_name':'category', 'event_type_1':'category', 'event_type_2':'category',
    'snap_CA':'category', 'snap_TX':'category', 'snap_WI':'category'
})

sales_long = compute_features(sales, prices, calendar)

cat_features = ['item_id', 'store_id',
            'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI',
            'is_year_end', 'is_year_start', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_weekend']

X_train, y_train, X_val, y_val = get_X_y(sales_long, cat_features)

model = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42, config_dict='TPOT light')

model.fit(X_train, y_train)

dump(model, 'tpot.joblib')
print(model.export())

### LightGBM



In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import lightgbm as lgb

# import sys
# sys.path.append('src')
from m5.global_model import get_X_and_y
from m5.hierarchy import compute_series_weights


data = Path('data/')
sales = pd.read_csv(data / 'sales_train_validation.csv', dtype={
    'item_id':'category', 'dept_id':'category', 'cat_id':'category', 'store_id':'category', 'state_id':'category'
})
prices = pd.read_csv(data / 'sell_prices.csv', dtype={
    'store_id': 'category', 'item_id': 'category'
})
calendar = pd.read_csv(data / 'calendar.csv', parse_dates=['date'], dtype={
    'weekday':'category', 'd':'category', 'event_name':'category', 'event_type_1':'category', 'event_type_2':'category',
    'snap_CA':'category', 'snap_TX':'category', 'snap_WI':'category'
})

In [1]:
cat_features = ['item_id', 'store_id',
            'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI',
            'is_year_end', 'is_year_start', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_weekend']

X_train, y_train, X_val, y_val = get_X_and_y(sales.iloc[:100, :], prices, calendar, cat_features)

In [1]:
y_plot = y_train.unstack('date').iloc[4, :52]
y_plot.index = y_plot.index.droplevel(0)
y_plot.to_frame().plot(figsize=(20,10))
X_train.unstack('date').loc[:, 'ewma_30'].iloc[4, :52].plot()

In [1]:
train_set = lgb.Dataset(X_train, y_train, categorical_feature = cat_features, free_raw_data=False)
val_set = lgb.Dataset(X_val, y_val, categorical_feature = cat_features, reference=train_set, free_raw_data=False)

In [1]:
params = {'metric': 'l2', 'objective': 'regression','seed': 23}

model = lgb.train(params, train_set, num_boost_round = 5000, early_stopping_rounds=100,
                  valid_sets = [train_set, val_set], verbose_eval = 100)
model.save_model('model.txt')

In [1]:
preds = model.predict(X_val, num_iteration=model.best_iteration)
y_hat_long = pd.DataFrame(preds, index=y_val.index, columns=['prediction'])
y_hat_long

### Scaled squared errors as custom loss



In [1]:
def get_y_weights(y: pd.Series, normalize=False):
    """
    For each series, compute the denominator in the MSSE loss function, i.e. the
    day-to-day variations squared, averaged by number of training observations.
    The weights can be normalized so that they add up to 1.
    This is provided to the lgb.Dataset for computing loss function and evaluation metric
    """
    scales = (y.unstack(level='date').diff(axis=1) ** 2).mean(axis=1)
    scales = scales.replace(0, pd.NA)
    weights = 1 / scales
    if normalize:
        weights = weights.divide(weights.sum())
    weights = y.merge(weights.to_frame('weight'), left_index=True, right_index=True)['weight']
    return weights

train_weights = get_y_weights(y_train)
val_weights = get_y_weights(y_val)

train_set = lgb.Dataset(X_train, y_train, weight=train_weights, categorical_feature = cat_features, free_raw_data=False)
val_set = lgb.Dataset(X_val, y_val, weight=val_weights, categorical_feature = cat_features, reference=train_set, free_raw_data=False)

def sse(preds, train_data):
    true = train_data.get_label()
    weights = train_data.get_weight() # weights is 1 / scale, normalized
    # loss = weights * (preds - true) ** 2
    gradient = weights * 2 * (preds - true)
    hessian = weights * 2
    return gradient, hessian

# model = lgb.train(params, train_set, num_boost_round = 5000, early_stopping_rounds=100,
#                   valid_sets = [train_set, val_set], verbose_eval = 100, fobj=sse)

### WRMSSE as custom metric



In [1]:
weights = compute_series_weights(sales, prices, calendar, groups={})

def get_wrmsse_lgb(preds, data):
    if preds.shape[0] == y_val.shape[0]:
        y = y_val.copy()
    else:
        y = y_train.copy()

    y['prediction'] = preds
    y['weight'] = data.get_weight() # weights is 1 / scale, normalized
    y['sse'] = ((y['sales'] - y['prediction']) ** 2) * y['weight']

    rmsse = y[['sse']].unstack(level='date').mean(axis=1).pow(1/2).to_frame(name='rmsse')
    rmsse =  rmsse.merge(weights['series_weight'], left_index=True, right_index=True)
    wrmsse = rmsse['rmsse'] * rmsse['series_weight']

    score = wrmsse.sum()
    return 'wrmsse', score, False

model = lgb.train(params, train_set, num_boost_round = 5000, early_stopping_rounds=100,
                  valid_sets = [train_set, val_set], verbose_eval = 100, fobj=sse, feval=get_wrmsse_lgb)
model.save_model('model_custom_loss.txt')

### Feature importance (and deselecting the unimportant ones)



In [1]:
from lightgbm import plot_importance
import matplotlib.pyplot as plt

model = lgb.Booster(model_file='data/models/model.txt')
ax = plot_importance(model, max_num_features=20, figsize=(20,10))
plt.show()
plt.savefig(data / 'lgbm_importance.png')

In [1]:
print("\n".join(("%s: %.2f" % x) for x in sorted(
    zip(train_sub[use_cols].columns, bst.feature_importance("gain")),
    key=lambda x: x[1], reverse=True  )))

In [1]:
import seaborn as sns
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importances_,X_val.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM feature importance')
plt.tight_layout()
plt.show()
plt.savefig(data / 'lgbm_importance.png')

### Comparing predictions



### train models for different horizons



### GridSearchCV



In [1]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)

In [1]:
lgb_grid = {'boosting_type' : ['gbdt'],
  'learning_rate' : [0.02, 0.03, 0.04, 0.05, 0.07, 0.1, 0.15, 0.2],
  'bagging_freq' : [1, 2, 3, 5, 10, 20, 50, 100],
  'feature_fraction' : [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.5, 0.6, 0.7,0.8,0.9,1.0],
  'bagging_fraction' : [0.2, 0.5, 0.7,0.8,0.85, 0.9, 0.95, 1.0],
  'max_depth' : [4,5,6,7,8,10,12,14,20],
  'num_leaves' : [7,15, 31, 63, 127, 255, 511, 1023, 2047],
  'min_data_in_leaf' : [1, 10, 20, 30, 50, 70, 100, 120, 150, 200, 300],
  'min_sum_hessian_in_leaf' : [0, 0.001, 0.01, 0.1, 1, 3, 10, 30, 100],
  'min_gain_to_split' : [0, 0.001, 0.01, 0.1, 1, 10, 100],
  'lambda_l1' : [0, 0.001, 0.01, 0.03, 0.1, 0.3, 1, 10, 100],
  'lambda_l2' : [0, 0.001, 0.01, 0.1, 1, 10, 100],
  'max_bin' : [63, 127, 255, 511, 1023, 2047],
  'use_onehot' : [True, False],
  'my_keep_mean_nan' : [True],
  'my_skip_first' : [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
  'my_log' : [True, False],
  'my_weights' : ['no', 'raw', 'norm'],
  'objective' : ['regression_l2','regression_l1','huber','fair'],
  'metric' : ['lgb_nwrmse']}

In [1]:
cv_results = lgb.cv(
        params,
        dftrainLGB,
        num_boost_round=100,
        nfold=3,
        metrics='mae',
        early_stopping_rounds=10,

        # This is what I added
        stratified=False
        )

## Evaluate forecast on all levels



In [1]:
y_hat = pd.read_hdf(data / 'y_hat_bottom_level.h5', 'y_hat_bottom_level')
sales_long = pd.read_hdf(data / 'sales_long.h5', key='sales_long')
weights = pd.read_hdf(data / 'weights_all_levels.h5', 'weights_all_levels')

In [1]:
bottom_level_wrmsse = get_wrmsse(y_hat, sales_long, weights)
bottom_level_wrmsse * 12

In [1]:
weights.loc[rmsse.index]['series_weight'] * rmsse

### RMSSE



$ \boldsymbol{R M S S E}=\sqrt{\frac{1}{h} \frac{\sum_{t=n+1}^{n+h}\left(Y_{t}-\bar{Y}_{t}\right)^{2}}{\frac{1}{n-1} \sum_{t=2}^{n}\left(Y_{t}-Y_{t-1}\right)^{2}}} $

-   Numerator: squared errors, averaged across all horizons
-   Denominator: day-to-day variations squared, averaged by number of training observations
-   The squared metric is then passed through sqrt



In [1]:
def get_rmsse(y_hat: pd.DataFrame, sales_long: pd.DataFrame):
    y_true = sales_long.loc[y_hat.index, y_hat.columns]
    y_train = sales_long.loc[sales_long.index.difference(y_hat.index), y_hat.columns]

    mse = ((y_hat - y_true) ** 2).mean(axis=0)
    scale = ((y_train.diff(axis=0)) ** 2).mean(axis=0)
    rmsse = np.sqrt(mse / scale)
    return rmsse

### WRMSSE



![img](/Users/luca/git/org/.attach/6D/F60BC8-A965-4C97-ADCA-549C5CAD7A48/_20200320_144903Screenshot 2020-03-20 at 14.48.58.png)

Once we have sales<sub>long</sub> and prices<sub>long</sub> in the same format, it should be
trivial to calculate for each series:

-   its dollar value in the last 28 days
-   the dollar value of its hierarchy in the last 28 days

Once computed, the weights can be persisted and reused



In [1]:
def get_wrmsse(y_hat: pd.DataFrame, sales_long: pd.DataFrame, weights: pd.DataFrame):
    rmsse = get_rmsse(y_hat, sales_long)
    wrmsse = (weights.loc[rmsse.index]['series_weight'] * rmsse).sum()
    return wrmsse

### Kaggle reference



In [1]:
from typing import Union

import numpy as np
import pandas as pd
from tqdm.auto import tqdm as tqdm

class WRMSSEEvaluator(object):

    group_ids = ( 'all_id', 'state_id', 'store_id', 'cat_id', 'dept_id', 'item_id',
        ['state_id', 'cat_id'],  ['state_id', 'dept_id'], ['store_id', 'cat_id'],
        ['store_id', 'dept_id'], ['item_id', 'state_id'], ['item_id', 'store_id'])

    def __init__(self,
                 train_df: pd.DataFrame,
                 valid_df: pd.DataFrame,
                 calendar: pd.DataFrame,
                 prices: pd.DataFrame):
        '''
        intialize and calculate weights
        '''
        self.calendar = calendar
        self.prices = prices
        self.train_df = train_df
        self.valid_df = valid_df
        self.train_target_columns = [i for i in self.train_df.columns if i.startswith('d_')]
        self.weight_columns = self.train_df.iloc[:, -28:].columns.tolist()

        self.train_df['all_id'] = "all"

        self.id_columns = [i for i in self.train_df.columns if not i.startswith('d_')]
        self.valid_target_columns = [i for i in self.valid_df.columns if i.startswith('d_')]

        if not all([c in self.valid_df.columns for c in self.id_columns]):
            self.valid_df = pd.concat([self.train_df[self.id_columns], self.valid_df],
                                      axis=1,
                                      sort=False)
        self.train_series = self.trans_30490_to_42840(self.train_df,
                                                      self.train_target_columns,
                                                      self.group_ids)
        self.valid_series = self.trans_30490_to_42840(self.valid_df,
                                                      self.valid_target_columns,
                                                      self.group_ids)
        self.weights = self.get_weight_df()

    def get_name(self, i):
        '''
        convert a str or list of strings to unique string
        used for naming each of 42840 series
        '''
        if type(i) == str or type(i) == int:
            return str(i)
        else:
            return "--".join(i)

    def get_weight_df(self) -> pd.DataFrame:
        '''
        returns weights for each of 42840 series in a dataFrame
        '''
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)
        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)
        weights_map = {}
        for i, group_id in enumerate(tqdm(self.group_ids, leave=False)):
            lv_weight = weight_df.groupby(group_id)[self.weight_columns].sum().sum(axis=1)
            lv_weight = lv_weight / lv_weight.sum()
            for i in range(len(lv_weight)):
                    weights_map[self.get_name(lv_weight.index[i])] = np.array([lv_weight.iloc[i]])
        weights = pd.DataFrame(weights_map).T / len(self.group_ids)

        return weights

    def trans_30490_to_42840(self, df, cols, group_ids):
        '''
        transform 30490 sries to all 42840 series
        '''
        series_map = {}
        for i, group_id in enumerate(tqdm(self.group_ids, leave=False)):
            tr = df.groupby(group_id)[cols].sum()
            for i in range(len(tr)):
                series_map[self.get_name(tr.index[i])] = tr.iloc[i].values
        return pd.DataFrame(series_map).T

    def get_rmsse(self, valid_preds) -> pd.Series:
        '''
        returns rmsse scores for all 42840 series
        '''
        score = ((self.valid_series - valid_preds) ** 2).mean(axis=1)
        scale = ((self.train_series.iloc[:, 1:].values - self.train_series.iloc[:, :-1].values) ** 2).mean(axis=1)
        rmsse = (score / scale).map(np.sqrt)
        return rmsse

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)
        valid_preds = self.trans_30490_to_42840(valid_preds, self.valid_target_columns, self.group_ids)
        self.rmsse = self.get_rmsse(valid_preds)
        self.contributors = pd.concat([self.weights, self.rmsse], axis=1, sort=False).prod(axis=1)
        return np.sum(self.contributors)

## Reconciliation



[https://otexts.com/fpp2/reconciliation.html](https://otexts.com/fpp2/reconciliation.html)
$$ \tilde{y}_h = S G \hat{y}_h $$, where $G$ is the &rsquo;reconciliation&rsquo; matrix
which minimizes the forecast errors of the base forecasts

Reconciliation approaches:

-   Bottom up (BU)
-   Top down:
    -   Average historical proportions (AHP)
    -   Proportions of historical averages (PHA)
    -   Forecast proportions (FP)
-   Middle-out
-   Optimal (minT):
    -   WLS, variance scaling
    -   WLS, structural scaling
    -   OLS



In [1]:
sum_matrix = compute_summing_matrix(sales)

n_bottom_nodes = sum_matrix.shape[1]
n_nodes = sum_matrix.shape[0]

### Structurally Weighted Least Squares



In [1]:
def optimalComb(forecastsDict, sumMat, method, mse_dict):
    if mse_dict is None:
        raise Exception('You cannot reconcile optimally without training errors')

    global optiMat
    hatMat = np.zeros([len(forecastsDict[0].yhat), 1])
    for key in forecastsDict.keys():
        f1 = np.array(forecastsDict[key].yhat)
        f2 = f1[:, np.newaxis]
        if np.all(hatMat == 0):
            hatMat = f2
        else:
            hatMat = np.concatenate((hatMat, f2), axis=1)
    ##
    # Multiply the Summing Matrix Together S*inv(S'S)*S'
    ##
    if method == "OLS":
        optiMat = np.dot(np.dot(sumMat, np.linalg.inv(np.dot(np.transpose(sumMat), sumMat))), np.transpose(sumMat))
    if method == "WLSS":
        """
        optimal combination by Structurally Weighted Least Squares
        This specification assumes that the bottom-level base forecast errors each have variance kh and are uncorrelated between nodes
        This estimator only depends on the structure of the aggregations, and not on the actual data
        """
        diagMat = np.diag(np.transpose(np.sum(sumMat, axis=1)))
        optiMat = np.dot(
            np.dot(np.dot(sumMat, np.linalg.inv(np.dot(np.dot(np.transpose(sumMat), np.linalg.inv(diagMat)), sumMat))),
                   np.transpose(sumMat)), np.linalg.inv(diagMat))
    if method == "WLSV":
        """
        optimal combination by Error Variance Weighted Least Squares
        This specification scales the base forecasts using the variance of the residuals
        """
        diagMat = [mse_dict[key] for key in mse_dict.keys()]
        diagMat = np.diag(np.flip(np.hstack(diagMat) + 0.0000001, 0))
        optiMat = np.dot(
            np.dot(np.dot(sumMat, np.linalg.inv(np.dot(np.dot(np.transpose(sumMat), np.linalg.inv(diagMat)), sumMat))),
                   np.transpose(sumMat)), np.linalg.inv(diagMat))

    newMat = np.empty([hatMat.shape[0], sumMat.shape[0]])
    for i in range(hatMat.shape[0]):
        newMat[i, :] = np.dot(optiMat, np.transpose(hatMat[i, :]))

    return newMat

### Reconciliation with htsprophet



In [1]:
import sys
sys.path.append('/Users/luca/git/htsprophet/htsprophet')
from htsprophet.__main__ import forecast_hts

#### Hierarchy nodes



In [1]:
from collections import OrderedDict
import pandas as pd


def get_hierarchy(sales: pd.DataFrame):
    hierarchy = OrderedDict()
    hierarchy['total->state_id'] = {'total': sales['state_id'].unique()}

    def fill_level(level, level_child):
        hierarchy[level + '->' + level_child] = {}
        for unique in sales[level].unique():
            hierarchy[level + '->' + level_child][unique] = sales[sales[level] == unique][level_child].unique()
        return hierarchy

    fill_level('state_id', 'store_id')
    fill_level('store_id', 'cat_id')
    fill_level('store_id', 'dept_id')

    return hierarchy


def get_hierarchy_nodes(hierarchy):
    nodes = []
    for level, level_dict in hierarchy.items():
        level_nodes = []
        for level_child, level_child_values in level_dict.items():
            level_nodes.append(level_child_values.size)
        nodes.append(level_nodes)

    return nodes

#### forecast and reconcile with htsprophet



In [1]:
hierarchy = get_hierarchy(sales)
nodes = get_hierarchy_nodes(hierarchy)
nodes

In [1]:
holidays = (calendar[['date', 'event_name_1']]
                .dropna()
                .reset_index(drop=True)
                .rename(columns={'date': 'ds', 'event_name_1': 'holiday'}))
holidays["lower_window"] = -4
holidays["upper_window"] = 3

forecasts_dict = forecast_hts(y=sales_long, h=28, nodes=nodes, holidays=holidays, method = "FP", daily_seasonality=False)

In [1]:
forecasts_dict

In [1]:
forecast_hts(y=forecasts_dict, skipFitting=True, h=28, nodes=nodes, holidays=holidays, method = "PHA", daily_seasonality=False)

In [1]:
for key in forecasts_dict.keys():
    print(key)

#### forecast with pmdarima, reconcile with htsprophet



$$ \tilde{y}_h = S G \hat{y}_h $$, where $G$ is the &rsquo;reconciliation&rsquo; matrix
which minimizes the forecast errors of the base forecasts

Reconciliation approaches:

-   Bottom up (BU)
-   Top down:
    -   Average historical proportions (AHP)
    -   Proportions of historical averages (PHA)
    -   Forecast proportions (FP)
-   Middle-out
-   Optimal (minT):
    -   WLS, variance scaling
    -   WLS, structural scaling
    -   OLS



In [1]:
reconciled_forecast = forecast_hts(y=y_pred, skipFitting=True, h=28, nodes=nodes, method = "PHA", daily_seasonality=False)

### Reconciliation with MLSE estimator



Pierre and Chiara&rsquo;s latest research on reconciliation:
[http://www.efc.pwr.edu.pl/assets/EFC19/Modica_EFC19.pdf](http://www.efc.pwr.edu.pl/assets/EFC19/Modica_EFC19.pdf)



#### VAR model



Intro to multivariate least squares:
[https://data.library.virginia.edu/getting-started-with-multivariate-multiple-regression/](https://data.library.virginia.edu/getting-started-with-multivariate-multiple-regression/)

When fitting a linear model on more than one dependent variables, instead of a
weight vectore we get a weight matrix, one row for each dependent variable.

If we run separate univariate models and one multivariate model we end up with
the same coefficients, the only difference is in the variance-covariance matrix
of the model coefficients.

The covariance matrix is then obtained from $\frac{1}{T-Q}(Y-\hat{A}Z) (Y-\hat{A}Z)'$ , where Q is the number of estimated parameters.

    set.seed(123) # Reset random number generator for reasons of reproducability
    
    # Generate sample
    t <- 200 # Number of time series observations
    k <- 2 # Number of endogenous variables
    p <- 2 # Number of lags
    
    # Generate coefficient matrices
    A.1 <- matrix(c(-.3, .6, -.4, .5), k) # Coefficient matrix of lag 1
    A.2 <- matrix(c(-.1, -.2, .1, .05), k) # Coefficient matrix of lag 2
    A <- cbind(A.1, A.2) # Companion form of the coefficient matrices
    
    # Generate series
    series <- matrix(0, k, t + 2*p) # Raw series with zeros
    for (i in (p + 1):(t + 2*p)){ # Generate series with e ~ N(0,0.5)
      series[, i] <- A.1%*%series[, i-1] + A.2%*%series[, i-2] + rnorm(k, 0, .5)
    }
    
    series <- ts(t(series[, -(1:p)])) # Convert to time series format
    names <- c("V1", "V2") # Rename variables
    
    plot.ts(series) # Plot the series

    library(vars) # Load package
    
    var.1 <- VAR(series, 2, type = "none") # Estimate the model
    vcov(var.1)



In [1]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

sales_train, sales_val = sales_long.iloc[:-28, :], sales_long.iloc[-28:, :]

scaler = scaler.fit(sales_train)
sales_train_norm = scaler.transform(sales_train)
sales_train_norm = pd.DataFrame(sales_train_norm, index=sales_train.index, columns=sales_train.columns)

In [1]:
from statsmodels.tsa.api import VAR

model = VAR(sales_train)
results = model.fit(2)

#### H matrix



$ \mathbf{H}^{\top}  =\left[\mathbf{I}_{\left(N-N_{L}\right)} \quad-\mathbf{A}\right] $

$$
\mathbf{x}=\mathbf{S x}^{*} \Longleftrightarrow \mathbf{H}^{\top} \mathbf{x}=\mathbf{0}
$$



In [1]:
x_matrix = np.dot(sum_matrix, sales_long.iloc[:, -n_bottom_nodes:].T)
assert all(x_matrix == sales_long.T)

In [1]:
a_matrix = sum_matrix.iloc[:-n_bottom_nodes, :].values
h_matrix = np.vstack((np.identity(a_matrix.shape[0]), -a_matrix.T))

assert (np.dot(h_matrix.T, sales_long.T)).all() == 0

#### Reconciled estimator



$$
\mathbf{C}_{k}=\mathbf{H}\left(\mathbf{H}^{\top} \mathbf{\Sigma}_{k} \mathbf{H}\right)^{-1} \mathbf{H}^{\top} \mathbf{\Sigma}_{k}
$$

$ \hat{\boldsymbol{\Theta}}_{k}^{\mathrm{MLSE}}=\hat{\boldsymbol{\Theta}}_{k}^{\mathrm{MLS}}\left(\mathbf{I}_{N}-\mathbf{C}_{k}\right) $



In [1]:
residuals_cov = results.sigma_u
# z = results.endog_lagged
# var_cov_matrix = np.kron(np.linalg.inv(z.T @ z), residuals_cov)
# var_cov_matrix.shape
theta = results.coefs

In [1]:
c_matrix = h_matrix @ np.linalg.inv(h_matrix.T @ residuals_cov @ h_matrix) @ h_matrix.T @ residuals_cov
proj_matrix = np.identity(n_nodes) - c_matrix

reconciled_theta = theta @ proj_matrix.values
results.coefs = reconciled_theta

In [1]:
y_hat = results.forecast(sales_train.values[-2:], 28)
y_hat = scaler.inverse_transform(y_hat)
y_hat = pd.DataFrame(y_hat, index=sales_val.index, columns=sales_val.columns)

### Reconciliation with fable



#### rpy2 magic



In [1]:
%load_ext rpy2.ipython

In [1]:
%%R
library(tsibble)
library(fable)
source('~/git/experiments/src/m5/reconcile.R')
# library(future)
# plan(multiprocess)

In [1]:
%%R -i sales_unp

sales_unp$date <- as.Date(sales_unp$date, format = "%y-%m-%d")

sales_ts <- sales_unp %>%
    as_tsibble(index=date, key = c(item_id, dept_id, cat_id, store_id, state_id))

sales_agg <- sales_ts %>%
    group_by(state_id) %%
        summarise(sales = sum(sales))
sales_agg

#### functions



    aggregate_sales <- function(dated_sales){
      dated_sales$date <- as.Date(dated_sales$date, format = "%y-%m-%d")
    
      sales_ts <- dated_sales %>%
        as_tsibble(index=date, key = c(item_id, dept_id, cat_id, store_id, state_id))
        ## fill_gaps(.full = TRUE)
    
      sales_agg <- sales_ts %>%
        aggregate_key((state_id / store_id / dept_id) * item_id, sales = sum(sales))
    }
    
    fit_model <- function(sales_agg, model_spec){
      ## model_spec <- ETS(sales ~ error("A") + trend("N") + season("N"), opt_crit = "mse")
      sales_fit <- sales_agg %>%
        ## filter_index("2016-01-01" ~ "2016-03-27")  %>%
        model(mod = model_spec)
    }
    
    forecast_reconciled <- function(sales_fit){
      sales_fc <- sales_fit  %>%
        reconcile(
          ## method = c("wls_var", "ols", "wls_struct", "mint_cov", "mint_shrink"),
          mod = min_trace(mod, method='mint_shrink')
        ) %>%
        forecast(h="28 days")
    }



\*\*



#### jupyter-R



    library(arrow)
    library(tsibble)
    library(fable)
    library(tidyverse)
    
    sales_unp <- read_parquet("/Users/luca/git/experiments/data/sales_unp.parquet", as_tibble=TRUE)
    sales_unp$date <- as.Date(sales_unp$date, format = "%y-%m-%d")
    
    sales_ts <- sales_unp %>%
        as_tsibble(index=date, key = c(item_id, dept_id, cat_id, store_id, state_id))

    sales_agg <- sales_ts %>%
        aggregate_key((state_id / store_id / dept_id) * item_id, sales=sum(sales))
    sales_agg

    source('~/git/experiments/src/m5/reconcile.R')
    
    sales_agg <- sales_ts %>% aggregate_sales()
    
    model_spec <- ETS(sales ~ error("A") + trend("N") + season("N"), opt_crit = "mse")
    sales_fit <- sales_agg %>% fit_model(model_spec)
    
    sales_fc <- sales_fit %>% forecast_reconciled()



## submit



### [[https://github.com/Kaggle/kaggle-api][Kaggle API]]



#### Credentials



We start by exporting `KAGGLE_USERNAME` and `KAGGLE_KEY` in `.envrc`. We can
find the credentials on kaggle.com after logging in.

We then install the kaggle API with `poetry add kaggle`



#### Download data



    cd ~/git/experiments/ &&
        poetry run kaggle competitions download -c m5-forecasting-accuracy -p data/

    unzip ~/git/experiments/data/m5-forecasting-accuracy.zip -d ~/git/experiments/data/



### Submission format



In [1]:
sample_submission = pd.read_csv(data / 'sample_submission.csv')
sample_submission.shape

The first half of the submission concerns the validation set, for days
1914-1941.
The second half concerns the test set, days 1942-1969. These will only be
evaluated in the last month so we just fill them with zeros.



### dummy submission



First dummy submission: just use the last 28 days in the train data!



In [1]:
df = pd.read_csv(data / 'sales_train_validation.csv')

submission = pd.read_csv(data / 'sample_submission.csv')

n_val_rows = int(submission.shape[0] / 2)

n_forecasted_dates = 28

submission.iloc[:n_val_rows, -n_forecasted_dates:] = df.iloc[:, -n_forecasted_dates:].values
submission.iloc[-n_val_rows:, -n_forecasted_dates:] = 1000

submission.to_csv(data / 'submission.csv', index=False)

### write submission file



In [1]:
submission = pd.read_csv(data / 'sample_submission.csv')

n_val_rows = int(submission.shape[0] / 2)
n_forecasted_dates = 28

submission.head()

In [1]:
submission.iloc[:n_val_rows, :] = df.iloc[:, -n_forecasted_dates:].values

submission.to_csv(data / 'submission.csv', index=False)

### Submit and see leaderboard



    cd ~/git/experiments/
    poetry run kaggle competitions submit -c m5-forecasting-accuracy -f data/submission.csv -m "Dummy submission"

    kaggle competitions submissions m5-forecasting-accuracy

fileName        date                 description       status    publicScore  privateScore
---------&#x2013;&#x2014;  --------------&#x2013;&#x2014;  -----------&#x2013;&#x2014;  ---&#x2013;&#x2014;  ------&#x2013;&#x2014;  -------&#x2013;&#x2014;
submission.csv  2020-03-06 12:27:04  Dummy submission  complete  1.15563      None

fileName        date                 description       status    publicScore  privateScore
---------&#x2013;&#x2014;  --------------&#x2013;&#x2014;  -----------&#x2013;&#x2014;  ---&#x2013;&#x2014;  ------&#x2013;&#x2014;  -------&#x2013;&#x2014;
submission.csv  2020-03-06 12:27:04  Dummy submission  complete  1.15563      None

    kaggle competitions leaderboard m5-forecasting-accuracy -s



## Interesting packages



-   AutoTS, support for Croston and hierarchical TS: [https://github.com/antoinecarme/pyaf](https://github.com/antoinecarme/pyaf)
    -   N-BEATS (winner of M4): [https://github.com/takotab/fastseq](https://github.com/takotab/fastseq)

